In [1]:
%run paths.ipynb
%run preprocess.ipynb
%run rouge.ipynb

**PARSE RAW DATA TO TEXT FILES**

In [2]:
p = parseNeuralSum()

In [3]:
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TRAIN_FOLDER, target_document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TRAIN_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TEST_FOLDER, target_document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TEST_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_VALIDATION_FOLDER, target_document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_VALIDATION_SUMMARY_FOLDER)

**CONVERT TEXT DATA TO WORD EMBEDDING IDS**

In [4]:
g = NeuralSumGlove(glove_filename=CONFIG.GLOVE_FILENAME)

In [5]:
g.convert_to_indices_(document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TRAIN_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TEST_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_VALIDATION_DUMP)

In [6]:
g.dump_all(CONFIG.GLOVE_CACHE_FILENAME)

**PAD EMBEDDING IDS DATA**

In [7]:
d = NeuralSumDataHandler()

In [8]:
d.load_dump(CONFIG.CNN_TRAIN_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.load_dump(CONFIG.CNN_TEST_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)

**COMPUTE ROUGE SCORES FOR REINFORCE TRAINING**

In [ ]:
%%time
r = RougeNeuralSum()
r.computeRefreshScores(CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, CONFIG.CNN_TRAIN_SUMMARY_FOLDER)
r.dump(CONFIG.CNN_TRAIN_ROUGE_DUMP)

**RUN FROM HERE FOR TRAINING**

In [9]:
%run paths.ipynb
%run preprocess.ipynb
%run model.ipynb
%run utils.ipynb
%run rouge.ipynb

In [10]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.make_batches(20)

g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)

r = RougeNeuralSum()
r.load(CONFIG.CNN_TRAIN_ROUGE_DUMP)

torch.cuda.set_device(0)

In [ ]:
m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
m.cuda()

EncoderDecoder(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400001, 100, padding_idx=400000)
    (sentenceEncoder): LSTM(100, 150, batch_first=True)
  )
  (documentEncoder): LSTM(150, 200, batch_first=True)
  (documentDecoder): LSTM(150, 200, batch_first=True)
  (classifier): Linear(in_features=200, out_features=1, bias=True)
)

In [ ]:
train_reinforce(m, batches=d.batches, lines=d.lines, scores=r.summary_scores, iterations=1)

In [ ]:
torch.save(m.state_dict(), '../parameters/best_params')

**RUN FROM HERE FOR TESTING**

In [ ]:
%run paths.ipynb
%run preprocess.ipynb
%run model.ipynb
%run utils.ipynb
%run rouge.ipynb

In [ ]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)
d.make_batches(20)
g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)

In [ ]:
#m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
#m.load_state_dict(torch.load('../parameters/parameters'))
#m.cuda()

In [ ]:
compute_accuracies(m, d.lines, d.output, d.batches)

In [ ]:
get_summaries(m, d.lines, d.batches, doc_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, write_folder=CONFIG.PREDICTED_SUMMARY_FOLDER)
output = computeRouge(CONFIG.PREDICTED_SUMMARY_FOLDER, CONFIG.CNN_TEST_SUMMARY_FOLDER)

In [ ]:
output[1]

In [ ]:
output[1]

In [ ]:
get_file(1001, CONFIG.PREDICTED_SUMMARY_FOLDER)

In [ ]:
get_file(1001, CONFIG.CNN_TEST_SUMMARY_FOLDER)